In [5]:
!pip install transformers open3d torch torchvision --user

^C


In [2]:
import cv2
import numpy as np
import open3d as o3d
from sklearn.neighbors import LocalOutlierFactor

def image_to_pointcloud(image_path, output_ply="output.ply"):
    """Convert a single image to a cleaned point cloud"""
    # 1. Read image and create synthetic depth (since OpenCV has no built-in depth estimation)
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    h, w = gray.shape
    
    # Create fake depth (darker pixels = farther away)
    depth = cv2.normalize(255 - gray.astype(np.float32), None, 0.1, 1.0, cv2.NORM_MINMAX)
    
    # 2. Generate 3D points
    points = []
    colors = []
    fx = fy = w * 0.8  # Focal length (approximate)
    cx, cy = w // 2, h // 2  # Optical center
    
    for v in range(0, h, 2):  # Skip every 2nd row for speed
        for u in range(0, w, 2):
            z = depth[v, u]
            if z < 0.15:  # Skip background
                continue
            x = (u - cx) * z / fx
            y = (v - cy) * z / fy
            points.append([x, y, z])
            colors.append(img[v, u] / 255.0)
    
    # 3. Clean outliers
    points = np.array(points)
    clf = LocalOutlierFactor(n_neighbors=20)
    inliers = clf.fit_predict(points) == 1
    points = points[inliers]
    colors = np.array(colors)[inliers]
    
    # 4. Create Open3D object
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.colors = o3d.utility.Vector3dVector(colors)
    
    # Save and visualize
    o3d.io.write_point_cloud(output_ply, pcd)
    o3d.visualization.draw_geometries([pcd])
    
    return pcd

# ACTUALLY RUN IT
point_cloud = image_to_pointcloud("chair.jpg")  # Replace with your image path

In [3]:
def prepare_for_gan(pcd, num_points=2048):
    """Convert Open3D point cloud to GAN-ready numpy array"""
    points = np.asarray(pcd.points)
    
    # 1. Normalize
    points -= np.mean(points, axis=0)
    points /= np.max(np.linalg.norm(points, axis=1))
    
    # 2. Fixed number of points
    if len(points) > num_points:
        points = points[np.random.choice(len(points), num_points, replace=False)]
    elif len(points) < num_points:
        points = points[np.random.choice(len(points), num_points, replace=True)]
    
    return points.astype(np.float32)

# Convert your generated point cloud
gan_input = prepare_for_gan(point_cloud)
print("GAN input shape:", gan_input.shape)  # Should be (2048, 3)

GAN input shape: (2048, 3)


In [ ]:
# Visualize the processed points
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(gan_input)
o3d.visualization.draw_geometries([pcd])

In [5]:
pip install opencv-python numpy open3d scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import cv2
import numpy as np
import os

def augment_image(img_path, output_folder="augmented_images"):
    """Generate 8 variations of your image"""
    os.makedirs(output_folder, exist_ok=True)
    img = cv2.imread(img_path)
    
    # List to store augmented images
    augmented = []
    
    # 1. Original
    augmented.append(img)
    cv2.imwrite(f"{output_folder}/original.jpg", img)
    
    # 2. Flipped horizontally
    flip_h = cv2.flip(img, 1)
    augmented.append(flip_h)
    cv2.imwrite(f"{output_folder}/flip_h.jpg", flip_h)
    
    # 3. Rotated +15 degrees
    M = cv2.getRotationMatrix2D((img.shape[1]//2, img.shape[0]//2), 15, 1)
    rot1 = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    augmented.append(rot1)
    cv2.imwrite(f"{output_folder}/rot15.jpg", rot1)
    
    # 4. Rotated -15 degrees
    M = cv2.getRotationMatrix2D((img.shape[1]//2, img.shape[0]//2), -15, 1)
    rot2 = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))
    augmented.append(rot2)
    cv2.imwrite(f"{output_folder}/rot-15.jpg", rot2)
    
    # 5. Brightened
    bright = cv2.convertScaleAbs(img, alpha=1.2, beta=30)
    augmented.append(bright)
    cv2.imwrite(f"{output_folder}/bright.jpg", bright)
    
    # 6. Darkened
    dark = cv2.convertScaleAbs(img, alpha=0.8, beta=-30)
    augmented.append(dark)
    cv2.imwrite(f"{output_folder}/dark.jpg", dark)
    
    return augmented

# Usage - replace "your_image.jpg" with your file
augment_image("cahir.jpg")

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:801: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'
